In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)
    
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Flatten, Dense, Dropout, \
GlobalMaxPooling2D, GlobalAveragePooling2D, Conv2D, MaxPooling2D, AveragePooling2D
from get_net import get_resnet50, get_VGG16, get_efficientnetB0, get_InceptionResNetV2, get_MobileNetV2, get_Xception, get_NASNetLarge,get_InceptionV3
from block import block
from image_generator import get_generator
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import os


filepath="checkpoint/model_{epoch:03d}-{val_loss:.5f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',verbose=1, 
                            save_best_only=True)
tbCallBack = TensorBoard(log_dir='logs',
                 histogram_freq=0,  
#                  batch_size=32,     
                 write_images=True,
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

#lrScheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1.0, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)


TRAIN_DATA_DIR  = 'all_samples_4000'

IMAGE_SIZE = (299,299,3)


NUM_CLASSES = 5


BATCH_SIZE = 16

FREEZE_LAYERS = 0


NUM_EPOCHS = 20


WEIGHTS_FINAL = 'model-inceptionv3-all-final.h5'

rotation_range=10
width_shift_range=0.1
height_shift_range=0.1
shear_range=0.2
zoom_range=0.2
channel_shift_range=0
horizontal_flip=True
vertical_flip=False
fill_mode='nearest'
validation_split=0.1
                                
train_generator, validation_generator = get_generator(TRAIN_DATA_DIR,
                                                      (IMAGE_SIZE[0],IMAGE_SIZE[1]),
                                                      BATCH_SIZE,
                                                      rotation_range,
                                                      width_shift_range,
                                                      height_shift_range,
                                                      shear_range,
                                                      zoom_range,
                                                      channel_shift_range,
                                                      horizontal_flip,
                                                      vertical_flip,
                                                      fill_mode,
                                                      validation_split)


def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(160,160,3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(5, activation='softmax'))
	# compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

#net_final = get_resnet50('imagenet',IMAGE_SIZE,FREEZE_LAYERS,NUM_CLASSES,0.5)
#net_final = get_efficientnetB0(IMAGE_SIZE,FREEZE_LAYERS,NUM_CLASSES,0.5)
net_final = get_InceptionV3(IMAGE_SIZE,FREEZE_LAYERS,NUM_CLASSES,0.5)
#net_final = define_model()
#net_final = load_model("model-resnet-all-final.h5")
print("no. of layers: "+str(len(net_final.layers)))

print(net_final.summary())

#opt = SGD(lr = 0.0001, decay = 1e-6, momentum = 0.9, nesterov = True)
#opt = Adam(lr=1e-3)
opt = RMSprop(lr=2e-5)
net_final.compile(optimizer=opt,
                  loss='categorical_crossentropy', metrics=['accuracy'])


net_final.fit_generator(train_generator,
                        steps_per_epoch = train_generator.samples // BATCH_SIZE,
                        validation_data = validation_generator,
                        validation_steps = validation_generator.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        callbacks = [checkpoint,tbCallBack])


net_final.save(WEIGHTS_FINAL)

Using TensorFlow backend.


Found 18000 images belonging to 5 classes.
Found 2000 images belonging to 5 classes.
Class #0 = drawings
Class #1 = hentai
Class #2 = neutral
Class #3 = porn
Class #4 = sexy
no. of layers: 315
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________


W0529 19:57:24.777655 13856 deprecation.py:323] From c:\users\jk\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1124/1125 [============================>.] - ETA: 0s - loss: 0.8174 - accuracy: 0.7049
Epoch 00001: val_loss improved from inf to 0.46323, saving model to checkpoint/model_001-0.46323.hdf5
1125/1125 [==============================] - 815s 724ms/step - loss: 0.8176 - accuracy: 0.7049 - val_loss: 0.4632 - val_accuracy: 0.8430
Epoch 2/20
1124/1125 [============================>.] - ETA: 0s - loss: 0.5570 - accuracy: 0.8063
Epoch 00002: val_loss improved from 0.46323 to 0.38900, saving model to checkpoint/model_002-0.38900.hdf5
1125/1125 [==============================] - 750s 667ms/step - loss: 0.5566 - accuracy: 0.8064 - val_loss: 0.3890 - val_accuracy: 0.8805
Epoch 3/20
1124/1125 [============================>.] - ETA: 0s - loss: 0.4787 - accuracy: 0.8368
Epoch 00003: val_loss did not improve from 0.38900
1125/1125 [==============================] - 720s 640ms/step - loss: 0.4787 - accuracy: 0.8368 - val_loss: 0.3898 - val_accuracy: 0.8785
Epoch 4/20
1124/1125 [=========================